In [1]:
# Imports
import numpy as np
import pandas as pd

In [ ]:
# data = pd.concat([pd.read_csv('Data/2009.csv'), pd.read_csv('Data/2010.csv'), 
#                   pd.read_csv('Data/2011.csv'), pd.read_csv('Data/2012.csv'), 
#                   pd.read_csv('Data/2013.csv'), pd.read_csv('Data/2014.csv'), 
#                   pd.read_csv('Data/2015.csv'), pd.read_csv('Data/2016.csv'), 
#                   pd.read_csv('Data/2017.csv'), pd.read_csv('Data/2018.csv')])

In [5]:
df = pd.read_csv("Data/2018.csv")
# Convert string to DateTime
df.FL_DATE = pd.to_datetime(df.FL_DATE, infer_datetime_format=True)
df = df.set_index('FL_DATE')

In [6]:
# Drop NaN values in the variables we will use in our baseline model
df = df[df['TAXI_OUT'].notna()]
df = df[df['ORIGIN'].notna()]
df = df[df['DEST'].notna()]


In [7]:
df.head()

,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
FL_DATE,,,,,,,,,,,,,,,,,,,,,
2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df.loc['2018-01-01':'2018-04-01']

In [16]:
df = pd.get_dummies(data=df, columns=['ORIGIN', 'DEST', 'OP_CARRIER'])

In [20]:
df = df.drop(['Unnamed: 27'], axis=1)

In [17]:
# Filter out early flights =< 0 DEP_DELAY
df = df[(df['CARRIER_DELAY'] >= 0)]

In [21]:
df.shape

(295894, 715)

In [22]:
# Save as new dataframe
df.to_csv('Data/data_2018.csv',index=False)